In [69]:
import os
import cv2
import numpy as np
from numpy import linalg as LA
import pandas as pd
import math
import traceback

In [49]:
# 中心座標を求める関数
def centerCoordinate(coordinates):
    value = 0
    for coor in coordinates:
        value += coordinates[coor]
    return value / 4

In [19]:
# 物体検出に必要な対応点の数の下限
MIN_MATCH_COUNT = 5

# テンプレート画像の読み込み
template = cv2.imread("/Users/yuki-f/Documents/SocSEL/Research/ImageRecognition/cat.png")

# 検索対象作品のスクリーンショットがあるディレクトリを指定
path = "/Users/yuki-f/Documents/SocSEL/Research/Selenium/dataset/sample/screenshots"

In [89]:
for pathName, dirName, fileNames in os.walk(path):
    fileLen = len([fileName for fileName in fileNames if not fileName.startswith(".")])
    
    # ディレクトリが空（スクショが存在しない）の場合はスキップ
    if fileLen == 0:
        continue
        
    prjId = pathName.rsplit("/")[len(pathName.rsplit("/")) - 1]
    
    coordinates = []
    
    # csvに出力するためのdataframeを作成
    df = pd.DataFrame(columns=["time", "x", "y"])
    
    for i in range(fileLen):
        fileName = pathName + "/" + prjId + "-" + str(i) + ".png"
        screenshot = cv2.imread(fileName)
        
        # 検出器生成
        sift = cv2.xfeatures2d.SIFT_create()
        
        # 画像の特徴量（kp:特徴点の座標情報等, des:特徴量記述子）
        try:
            kp_t,des_t = sift.detectAndCompute(template,None)
            kp_s,des_s = sift.detectAndCompute(screenshot,None)

            FLANN_INDEX_KDTREE = 0
            index_params = dict(algorithm=FLANN_INDEX_KDTREE,tress=5)
            search_params = dict(checks = 50)

            flann = cv2.FlannBasedMatcher(index_params, search_params)
            matches = flann.knnMatch(des_t,des_s,k=2)

            good = []
            for m,n in matches:
                if m.distance < 0.7 * n.distance:
                    good.append(m)

            # 対応点がMIN_MATCH_COUNT以上あれば物体を検出
            if len(good) > MIN_MATCH_COUNT:
                src_pts = np.float32([ kp_t[m.queryIdx].pt for m in good ]).reshape(-1,1,2)
                dst_pts = np.float32([ kp_s[m.trainIdx].pt for m in good ]).reshape(-1,1,2)

                M, mask = cv2.findHomography(src_pts, dst_pts, cv2.RANSAC,5.0)
                matchesMask = mask.ravel().tolist()

                h,w = template.shape[:2]
                pts = np.float32([ [0,0],[0,h-1],[w-1,h-1],[w-1,0] ]).reshape(-1,1,2)
                dst = cv2.perspectiveTransform(pts,M)

                screenshot = cv2.polylines(screenshot,[np.int32(dst)],True,255,3, cv2.LINE_AA)
                
                flattenCoors = np.int32(dst).flatten()
                
                # 検出した物体の四隅の座標
                coordinates.append(
                    {
                        "index": i,
                        "x": {
                            "upperLeft": flattenCoors[0], 
                            "lowerLeft": flattenCoors[2],
                            "lowerRight": flattenCoors[4],
                            "upperRight": flattenCoors[6]
                        },
                        "y": {
                            "upperLeft": flattenCoors[1], 
                            "lowerLeft": flattenCoors[3],
                            "lowerRight": flattenCoors[5],
                            "upperRight": flattenCoors[7]
                        }
                    }
                )
                
                addRow = pd.DataFrame([[i, centerCoordinate(coordinates[i]["x"]), centerCoordinate(coordinates[i]["y"])]])
                print(addRow)
                df = df.append([df, addRow], axis=1)
                print(df)
                
            else:
                print("マッチングなし")
                matchesMask = None
                            
        except Exception as e:
            print(traceback.format_exc())
            continue 

    if len(coordinates) <= 0:
        continue
    
    df.to_csv(prjId + ".csv")

   0      1      2
0  0  174.5  310.0
Traceback (most recent call last):
  File "<ipython-input-89-22d26588566a>", line 76, in <module>
    df = df.append([df, addRow], axis=1)
TypeError: append() got an unexpected keyword argument 'axis'

   0      1      2
0  1  262.5  310.0
Traceback (most recent call last):
  File "<ipython-input-89-22d26588566a>", line 76, in <module>
    df = df.append([df, addRow], axis=1)
TypeError: append() got an unexpected keyword argument 'axis'

   0      1      2
0  2  334.5  310.0
Traceback (most recent call last):
  File "<ipython-input-89-22d26588566a>", line 76, in <module>
    df = df.append([df, addRow], axis=1)
TypeError: append() got an unexpected keyword argument 'axis'

   0      1      2
0  3  406.5  310.0
Traceback (most recent call last):
  File "<ipython-input-89-22d26588566a>", line 76, in <module>
    df = df.append([df, addRow], axis=1)
TypeError: append() got an unexpected keyword argument 'axis'

   0      1      2
0  4  430.5  262.0
Tr